# Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
import math 

# Define Functions 

In [2]:
import math

def median(lst):
    return np.median(np.array(lst)) 

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def Avg_Exp_lines(img, lines, color=[0, 255, 0], thickness=5) :
    """
    polyfit + extrapolation
    """
    
    Y_pixels = img.shape[0]
    X_pixels = img.shape[1]
    
    xR = []
    yR = []
    mR = []
    xL = []
    yL = []
    mL = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = ((y2-y1)/(x2-x1)) 
            if slope > 0 and x1>X_pixels/2 and x2>X_pixels/2 :  # right line (opposite to Cartesian)
                xR.extend((x1,x2))
                yR.extend((y1,y2))
                mR.extend([slope])
            elif slope < 0 and x1<X_pixels/2 and x2<X_pixels/2 :
                xL.extend((x1,x2))
                yL.extend((y1,y2))
                mL.extend([slope])
    
    R_fit = np.polyfit(xR,yR,1)
    L_fit = np.polyfit(xL,yL,1)
    
    mR_fit = R_fit[0]
    cR_fit = R_fit[1]
    mL_fit = L_fit[0]
    cL_fit = L_fit[1]
    
    xRmin,xRmax = min(xR), max(xR) 
    xLmin,xLmax = min(xL), max(xL)
    
    yRmax = int(xRmax*mR_fit+cR_fit)
    yRmin = int(xRmin*mR_fit+cR_fit)
    
    if yRmax < Y_pixels -10:
        yRmax = Y_pixels
        xRmax = int((yRmax-cR_fit)/mR_fit)
    if yRmin > Y_pixels/2:
        yRmin = int(Y_pixels/2+55)
        xRmin = int((yRmin-cR_fit)/mR_fit)
    
    yLmax = int(xLmin*mL_fit+cL_fit) 
    yLmin = int(xLmax*mL_fit+cL_fit)
    
    if yLmax < Y_pixels -10:
        yLmax = Y_pixels
        xLmin = int((yLmax-cL_fit)/mL_fit)
    if yLmin > Y_pixels/2:
        yLmin = int(Y_pixels/2+55)
        xLmax = int((yLmin-cL_fit)/mL_fit)
        
    line_img2 = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    cv2.line(line_img2, (xRmin, yRmin), (xRmax, yRmax), color, thickness)
    
    cv2.line(line_img2, (xLmin, yLmax), (xLmax, yLmin), color, thickness)
    
    return line_img2
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_all_image(instant_image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # Define grsyscale image
    gray = grayscale(instant_image)

    # Define a kernel size and apply Gaussian smoothing
    # kernel_size = 5
    blur_gray = gaussian_blur(gray,kernel_size)
    
    # Define our parameters for Canny and apply
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    #vertices = np.array([[(0,imshape[0]),(imshape[1]/2-small_x, imshape[0]/2+small_y), (imshape[1]/2+small_x, imshape[0]/2+small_y), (imshape[1],imshape[0])]], dtype=np.int32)
    masked = region_of_interest(edges,vertices)
    
    H_image, H_lines = hough_lines(masked, rho, theta, threshold, min_line_len, max_line_gap)
    
    H_image2 = Avg_Exp_lines(masked, H_lines)
    
    final_img = weighted_img(H_image, instant_image, α=0.8, β=1., λ=0.)
    final_img2 = weighted_img(H_image2, instant_image, α=0.8, β=1., λ=0.)
    
    return blur_gray, edges, masked, H_image, final_img, final_img2

def process_image(instant_image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # Define grsyscale image
    gray = grayscale(instant_image)

    # Define a kernel size and apply Gaussian smoothing
    # kernel_size = 5
    blur_gray = gaussian_blur(gray,kernel_size)
    
    # Define our parameters for Canny and apply
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    #vertices = np.array([[(0,imshape[0]),(imshape[1]/2-small_x, imshape[0]/2+small_y), (imshape[1]/2+small_x, imshape[0]/2+small_y), (imshape[1],imshape[0])]], dtype=np.int32)
    masked = region_of_interest(edges,vertices)
    
    H_image, H_lines = hough_lines(masked, rho, theta, threshold, min_line_len, max_line_gap)
    
    H_image2 = Avg_Exp_lines(masked, H_lines)
    
    final_img2 = weighted_img(H_image2, instant_image, α=0.8, β=1., λ=0.)
    
    return final_img2

# Import Images in 'test_images' 

In [3]:
import os
img_files = os.listdir("test_images/")
print(img_files)

current_image = img_files[5]
print(current_image)
#reading in an image
image = mpimg.imread('test_images/'+ current_image)

copied_image = image 
Y_pixels = copied_image.shape[0]
X_pixels = copied_image.shape[1]
print('This image is:', type(copied_image), 'with dimensions:', copied_image.shape)



['solidWhiteCurve.jpg', 'solidWhiteRight.jpg', 'solidYellowCurve.jpg', 'solidYellowCurve2.jpg', 'solidYellowLeft.jpg', 'whiteCarLaneSwitch.jpg']
whiteCarLaneSwitch.jpg
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


# Define Parameters for Image Processes

In [4]:
# Gaussian Blur
kernel_size = 5

# Canny Function
low_threshold = 70
high_threshold = 200

# Masked area
small_x = 5
small_y = 47
vertices = np.array([[(25,Y_pixels),(X_pixels/2-small_x, Y_pixels/2+small_y), (X_pixels/2+small_x, Y_pixels/2+small_y), (X_pixels,Y_pixels)]], dtype=np.int32)

# Hough Function
rho = 1           # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 1     # minimum number of votes (intersections in Hough grid cell)
min_line_len = 15 # minimum number of pixels making up a line
max_line_gap = 20    # maximum gap in pixels between connectable line segments

# Process Image and Save in 'test_images_output'

In [ ]:
#plt.imshow(copied_image)  
gray, edges, mask, H_image, lane_lines, lane_solid = process_all_image(copied_image)

plt.imsave('test_images_output/'+ current_image, copied_image)
plt.imsave('test_images_output/'+ current_image[:-4] +'_blurgray.jpg', gray, cmap='gray')
plt.imsave('test_images_output/'+ current_image[:-4] +'_canny.jpg', edges)
plt.imsave('test_images_output/'+ current_image[:-4] +'_mask.jpg', mask)
plt.imsave('test_images_output/'+ current_image[:-4] +'_Hough.jpg', lane_lines)
plt.imsave('test_images_output/'+ current_image[:-4] +'_Solid_Lanes.jpg', lane_solid)

# Time for the Videos 
## White

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'

clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Yellow

In [6]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(20,25)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:17<00:00, 39.05it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 18.1 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Challenge (not finished)

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))